# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Описание данных

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.

При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

In [1]:
import pandas as pd 

In [2]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

Загружаем и смотрим на данные 

In [3]:
df_list = {'0' : df_0, # создаем словарь для последующего цикла с перебором по ключам=датафреймам 
           '1' : df_1,
           '2' : df_2}

for i in df_list:
    df_list[i].info()
    display(df_list[i].head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


В данных видими 3 файла с данными:
- 4 столбца
- 1000 стр в каждом датафрейме 
- типы данных менять не нужно, т.к. все нужные нам для дадльнейшей работы столбцы типа float 

Проверим пропуски и дупликаты 

In [4]:
def duplicates(df): 
    duplicates = df.duplicated().sum()
    if duplicates == 0:
        print('Дупликатов нет')
    else:
        print(f'Дупликатов:{duplicates}')
        
for i in df_list.values():
    duplicates(i)

Дупликатов нет
Дупликатов нет
Дупликатов нет


In [5]:
def missing_values(df):
    isna = df.isna().sum().sum()
    if isna == 0:
        print('Пропусков нет')
    else:
        print(f'Пропусков:{isna}')
        
for i in df_list.values():
    missing_values(i)

Пропусков нет
Пропусков нет
Пропусков нет


Убираем лишний столбец который не понадобиться в обучении модели

In [6]:
df_0.drop(['id'], axis = 1, inplace = True)
df_1.drop(['id'], axis = 1, inplace = True)
df_2.drop(['id'], axis = 1, inplace = True)

***Итоги:***
- Проверили данные на пропуски и дупликаты 
- Подготовили данные к обучении модели

## Обучение и проверка модели

Импортируем нужные библиотеки и обучаем модели линейной регрессии + сохраняем в переменные результаты 

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [8]:
def liner_regression (data):
    df_features = data.drop(['product'], axis=1)
    df_target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(df_features, 
                                                                                  df_target, 
                                                                                  test_size = 0.25,
                                                                                  random_state = 123)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    predictions_valid_mean = predictions_valid.mean()
    result = mean_squared_error(target_valid, predictions_valid, squared=False)
    return target_valid, predictions_valid, predictions_valid_mean, result

target_valid_0, predictions_valid_0, predictions_valid_mean_0, result_0 = liner_regression(df_0)
target_valid_1, predictions_valid_1, predictions_valid_mean_1, result_1 = liner_regression(df_1)
target_valid_2, predictions_valid_2, predictions_valid_mean_2, result_2 = liner_regression(df_2)

print("Регион №1")
print("Cредний запас предсказанного сырья:", predictions_valid_mean_0)
print("RMSE модели линейной регрессии на валидационной выборке:", result_0)

print("\nРегион №2")
print("Cредний запас предсказанного сырья:", predictions_valid_mean_1)
print("RMSE модели линейной регрессии на валидационной выборке:", result_1)

print("\nРегион №3")
print("Cредний запас предсказанного сырья:", predictions_valid_mean_2)
print("RMSE модели линейной регрессии на валидационной выборке:", result_2)

Регион №1
Cредний запас предсказанного сырья: 92.54936189116306
RMSE модели линейной регрессии на валидационной выборке: 37.64786282376177

Регион №2
Cредний запас предсказанного сырья: 69.28001860653976
RMSE модели линейной регрессии на валидационной выборке: 0.8954139804944313

Регион №3
Cредний запас предсказанного сырья: 95.09859933591373
RMSE модели линейной регрессии на валидационной выборке: 40.12803006598514


***Итоги:***

Результаты обучения и проверки регрессионных моделей:

*1-й регион:*
- Cредний запас предсказанного сырья: 92.54936189116306
- RMSE модели линейной регрессии на валидационной выборке: 37.64786282376177

*2-й регион:*
- Cредний запас предсказанного сырья: 69.28001860653976
- RMSE модели линейной регрессии на валидационной выборке: 0.8954139804944313

*3-й регион:*
- Cредний запас предсказанного сырья: 95.09859933591373
- RMSE модели линейной регрессии на валидационной выборке: 40.12803006598514
    
Лучший RMSE на валидационной выборке на 2-ом регионе.
    

## Подготовка к расчёту прибыли

Расчитаем достаточный объём сырья для безубыточной разработки новой скважины и сравним полученный объём сырья со средним запасом в каждом регионе. 

Сначала считаем бюджет для каждой скважины из 200 лучших, а затем делим этот бюджет на прибыль каждой скважины чтобы узнать сколько должно быть скважин для минимального покрытия бюджета в 0

In [9]:
budget = 10_000_000_000
income_per_unit = 450_000
count = 200
all_bores = 500

budget_per_well = budget / count
min_value = budget_per_well / income_per_unit

print(f'Безубыточность: {min_value}')

print("\nРегион №1")
print("Cредний запас предсказанного сырья:", predictions_valid_mean_0)

print("\nРегион №2")
print("Cредний запас предсказанного сырья:", predictions_valid_mean_1)

print("\nРегион №3")
print("Cредний запас предсказанного сырья:", predictions_valid_mean_2)

Безубыточность: 111.11111111111111

Регион №1
Cредний запас предсказанного сырья: 92.54936189116306

Регион №2
Cредний запас предсказанного сырья: 69.28001860653976

Регион №3
Cредний запас предсказанного сырья: 95.09859933591373


***Итоги:***

Как видно из предсказанных запасов по всем регионам:

- 1-й регион: Cредний запас предсказанного сырья: 92.54936189116306

- 2-й регион: Cредний запас предсказанного сырья: 69.28001860653976

- 3-й регион: Cредний запас предсказанного сырья: 95.09859933591373


,ни один из них не может быть безубыточным, т.к. запасы сырья у всех регионов меньше 111.11111111111111  

## Расчёт прибыли и рисков 

Создадим функцию для подчета прибыли 

In [10]:
def calculate_profit(predictions, target):
    predictions = pd.Series(predictions, index=target.index) # преобразем в Series вместо numpyarray
    pred_sorted = predictions.sort_values(ascending=False) # сортируем от большего к меньшему для лучших результатов
    selected = target[pred_sorted.index][:count] # выбираем наибольшие обьемы сырья 200 лучших 
   
    revenue = selected.sum() * income_per_unit  
    profit = revenue - budget  

    return profit

profit_0 = calculate_profit(predictions_valid_0, target_valid_0)
profit_1 = calculate_profit(predictions_valid_1, target_valid_1)
profit_2 = calculate_profit(predictions_valid_2, target_valid_2)

print('Прибыль в регионе №1:', profit_0)
print('Прибыль в регионе №2:', profit_1)
print('Прибыль в регионе №3:', profit_2) 

Прибыль в регионе №1: 3534670917.261383
Прибыль в регионе №2: 2415086696.681511
Прибыль в регионе №3: 2370343863.0213737


Создадим функцию для bootstrap, которая оценит стандартные ошибки и доверительные интервалы для 3 регионов.  

In [15]:
import numpy as np
def profit_bootstrap(target, predictions):
    state = np.random.RandomState(123)
    values = []

    for i in range(1000):
        target_sample = target.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        predictions_sample = predictions[target_sample.index]
        value = calculate_profit(predictions_sample, target_sample)
        values.append(value)

    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean_profit = values.mean()
    risk = (values < 0).mean()

    return mean_profit, lower, upper, risk

mean_profit_0, lower_0, upper_0, risk_0 = profit_bootstrap(target_valid_0, predictions_valid_0)
mean_profit_1, lower_1, upper_1, risk_1 = profit_bootstrap(target_valid_1, predictions_valid_1)
mean_profit_2, lower_2, upper_2, risk_2 = profit_bootstrap(target_valid_2, predictions_valid_2)

print('Регион №1:')
print(f'Средняя прибыль: {mean_profit_0}')
print(f'Доверительный интервал: {lower_0}, {upper_0}')
print(f'Риск убытков: {risk_0:.1%}')

print('\nРегион №2:')
print(f'Средняя прибыль: {mean_profit_1}')
print(f'Доверительный интервал: {lower_1}, {upper_1}')
print(f'Риск убытков: {risk_1:.1%}')

print('\nРегион №3:')
print(f'Средняя прибыль: {mean_profit_2}')
print(f'Доверительный интервал: {lower_2}, {upper_2}')
print(f'Риск убытков: {risk_2:.1%}')

Регион №1:
Средняя прибыль: 504778653.6473463
Доверительный интервал: -51833206.45118432, 1029933076.4951894
Риск убытков: 4.0%

Регион №2:
Средняя прибыль: 538879609.1841167
Доверительный интервал: 104974008.53059308, 963564955.3708618
Риск убытков: 0.5%

Регион №3:
Средняя прибыль: 364222305.2978538
Доверительный интервал: -220913808.22524208, 906489773.8374137
Риск убытков: 9.5%


***Итоги:***

Как видно из расчета прибыли и рисков по всем регионам мы имеем следующую картину:

1-й регион: 
- Риск убыточности - 4%

2-й регион: 
- Риск убыточности - 0.5%. 

3-й регион: 
- Риск убыточности - 9.5%. 

Исходя из полученных данных, самым перпективным для разработки регионом будет ***регион №2***. т.к. он имеет наибольшую среднюю прибыль, а также наименьшую вероятность убыточности.